In [30]:
!pip install uv && uv pip install transformers datasets polars spacy gliner spacy-curated-transformers spacy-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.9.20 environment at /opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo
Resolved 127 packages in 9.51s                                       
Building seqeval==1.2.2                                                
Building seqeval==1.2.2                                        
Building intervaltree==3.1.0
Building seqeval==1.2.2                                        
Building intervaltree==3.1.0
Building docopt==0.6.2
Building seqeval==1.2.2                                        
Building intervaltree==3.1.0
Building docopt==0.6.2
Building sqlitedict==2.1.0
Building seqeval==1.2.2                                        
Building intervaltree==3.1.0
Building docopt==0.6.2
Building sqlitedict==2.1.0
Building wikipedia-api==0.7.1
Building seqeval==1.2.2                                        
Building intervaltree==3.1.0
Building docopt==0.6.2
Building sqlitedict==2.1.0
Building wikipedia-api==0.7.1
Building pptree==3.1
Building seqeval==1.2.2                            

In [ ]:
!python -m spacy download en_core_web_trf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 20.6 MB/s eta 0:00:0000:0100:01
  Using cached curated_transformers-0.1.1-py2.py3-none-any.whl.metadata (965 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 4.0 MB/s eta 0:00:00-:--:--
Using cached curated_transformers-0.1.1-py2.py3-none-any.whl (25 kB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [6]:
import polars as pl

In [7]:
import polars as pl

splits = {'validation': 'v1.1/validation-00000-of-00001.parquet', 'train': 'v1.1/train-00000-of-00001.parquet', 'test': 'v1.1/test-00000-of-00001.parquet'}
df = pl.read_parquet('hf://datasets/microsoft/ms_marco/' + splits['train'])


In [8]:
df["query_type"].value_counts()

query_type,count
str,u32
"""numeric""",22758
"""person""",2026
"""description""",44961
"""entity""",8529
"""location""",4052


In [9]:
# Show preview of each query type, 10 rows

for query_type in df["query_type"].unique().to_list():
    print(f"Query Type: {query_type}")
    print(df.filter(pl.col("query_type") == query_type).head(10))

Query Type: person
shape: (10, 6)
┌──────────────────┬──────────────────┬──────────────────┬──────────┬────────────┬─────────────────┐
│ answers          ┆ passages         ┆ query            ┆ query_id ┆ query_type ┆ wellFormedAnswe │
│ ---              ┆ ---              ┆ ---              ┆ ---      ┆ ---        ┆ rs              │
│ list[str]        ┆ struct[3]        ┆ str              ┆ i32      ┆ str        ┆ ---             │
│                  ┆                  ┆                  ┆          ┆            ┆ list[str]       │
╞══════════════════╪══════════════════╪══════════════════╪══════════╪════════════╪═════════════════╡
│ ["Dr. Seuss"]    ┆ {[0, 0, …        ┆ remembering the  ┆ 19710    ┆ person     ┆ []              │
│                  ┆ 1],["For the TV  ┆ name of the au…  ┆          ┆            ┆                 │
│                  ┆ spec…            ┆                  ┆          ┆            ┆                 │
│ ["In a New York  ┆ {[1, 0, …        ┆ who coined the   

In [59]:
nlp = spacy.load(
    "en_core_web_sm",
    # enable=["parser", "ner"], 
    disable=["textcat", "lemmatizer"],
)
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [ ]:
from dataclasses import dataclass
from typing import List


@dataclass
class QuestionComponents:
    wh_word: str
    root_verb: str
    subject: str
    objects: List[str]
    modifiers: List[str]


class FastDependencyParser:
    def __init__(self, nlp=None):
        # Load smallest model for speed
        # Common question patterns
        self.wh_patterns = {
            "what": {"definition", "description", "purpose"},
            "when": {"time", "date", "period"},
            "where": {"location", "place", "direction"},
            "who": {"person", "agent", "doer"},
            "why": {"reason", "cause", "purpose"},
            "how": {"method", "manner", "degree"},
        }
        self.nlp = nlp

    def extract_components(self, doc) -> QuestionComponents:
        """Fast extraction of question components using deps"""
        wh_word = ""
        root_verb = ""
        subject = ""
        objects = []
        modifiers = []

        for token in doc:
            if token.dep_ == "ROOT":
                root_verb = token.text
            elif token.tag_.startswith("W"):
                wh_word = token.text.lower()
            elif token.dep_ in ("nsubj", "nsubjpass"):
                subject = token.text
            elif token.dep_ in ("dobj", "pobj", "attr"):
                objects.append(token.text)
            elif token.dep_ in ("advmod", "amod"):
                modifiers.append(token.text)

        return QuestionComponents(
            wh_word=wh_word,
            root_verb=root_verb,
            subject=subject,
            objects=objects,
            modifiers=modifiers,
        )


class DependencyBasedReformulator:
    def __init__(self, nlp):
        self.parser = FastDependencyParser(nlp)

        # Load minimal word vectors for key terms
        self.word_vectors = self._load_minimal_vectors()

    def _load_minimal_vectors(self):
        """Load only essential vectors for question words and common verbs"""
        return {
            # Question words and similar terms
            "what": ["define", "explain", "describe"],
            "when": ["time", "date", "period"],
            "where": ["location", "place", "area"],
            "who": ["person", "individual"],
            # Common verbs and alternatives
            "is": ["was", "are", "exists"],
            "do": ["does", "did", "perform"],
            "has": ["have", "had", "possess"],
        }

    def reformulate(self, query: str) -> List[str]:
        # 1. Parse query (~5ms)
        doc = self.parser.nlp(query)
        components = self.parser.extract_components(doc)

        # 2. Generate structural variations (~2ms)
        base_variations = self._generate_structural_variations(components)

        return base_variations

        # # 3. Apply word substitutions (~2ms)
        # semantic_variations = self._apply_word_substitutions(base_variations)

        # return list(set(semantic_variations))[:2]

    def _generate_structural_variations(self, comp: QuestionComponents) -> List[str]:
        """Generate variations based on dependency structure"""
        variations = []

        # Basic structure patterns
        if comp.wh_word == "what" and comp.root_verb in ("is", "are"):
                # Definition pattern
                variations.extend(
                    [
                        f"{comp.subject} definition",
                        f"{comp.subject} meaning",
                        f"{comp.subject} explanation",
                        f"{comp.subject} basics",
                    ]
                )

        elif comp.wh_word == "when":
            # Temporal patterns
            event = " ".join([comp.subject, comp.root_verb])
            variations.extend([f"time of {event}", f"date of {event}"])

        elif comp.wh_word == "where":
            # Location patterns
            entity = comp.subject
            variations.extend([f"location of {entity}", f"find {entity}"])
        elif comp.wh_word == "who":
            # Person patterns
            action = " ".join([comp.root_verb, comp.objects[-1]])
            variations.extend([f"{action}"])
        else:
            variations.extend(
                [
                    f"{comp.subject} {comp.root_verb} {' '.join(comp.objects)}",
                ]
            )

        return variations

    def _apply_word_substitutions(self, variations: List[str]) -> List[str]:
        """Apply word vector based substitutions"""
        result = set()

        for var in variations:
            words = var.split()
            for i, word in enumerate(words):
                if word in self.word_vectors:
                    for similar in self.word_vectors[word]:
                        new_words = words.copy()
                        new_words[i] = similar
                        result.add(" ".join(new_words))

        return list(result)


In [64]:

# Example usage
reformulator = DependencyBasedReformulator(nlp)

test_queries = [
    "What is machine learning?",
    "When did World War 2 end?",
    "Where is the Eiffel Tower located?",
    "Who invented the telephone?",
    "How does photosynthesis work?",
    "what are some ways to do fast query reformulation"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    print("Reformulations:", reformulator.reformulate(query))


Query: What is machine learning?
Reformulations: ['learning definition', 'learning meaning', 'learning explanation', 'learning basics']

Query: When did World War 2 end?
Reformulations: ['time of World War 2 did', 'date of World War 2 did']

Query: Where is the Eiffel Tower located?
Reformulations: ['location of the Eiffel Tower', 'find the Eiffel Tower']

Query: Who invented the telephone?
Reformulations: ['invented the telephone']

Query: How does photosynthesis work?
Reformulations: [' How does photosynthesis work ']

Query: what are some ways to do fast query reformulation
Reformulations: ['some ways definition', 'some ways meaning', 'some ways explanation', 'some ways basics']


In [62]:
%time reformulator.parser.extract_components(nlp("what are some ways to do fast query reformulation?"))

CPU times: user 5.78 ms, sys: 2.65 ms, total: 8.44 ms
Wall time: 6.33 ms


QuestionComponents(wh_word='what', root_verb='are', subject='some ways', objects=['fast query reformulation'], modifiers=[])

In [39]:
reformulator.parser.extract_components(nlp("What is machine learning?"))

QuestionComponents(wh_word='what', root_verb='is', subject='machine learning', objects=[], modifiers=[])

In [40]:
!pip install -U diaparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00


In [ ]:
from diaparser.parsers import Parser
parser = Parser.load('en_ewt-electra')


Downloading: "https://github.com/Unipisa/diaparser/releases/download/v1.0/en_ewt.electra-base" to /Users/nhuantran/.cache/diaparser/en_ewt.electra-base
100%|██████████| 452M/452M [00:35<00:00, 13.4MB/s] 
/opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be re

In [49]:
%time dataset = parser.predict(["what are some ways to do fast query reformulation".split()], prob=True)

CPU times: user 183 ms, sys: 60.4 ms, total: 244 ms
Wall time: 50.2 ms


/opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo/lib/python3.9/site-packages/diaparser/modules/lstm.py:86: FutureWarning: `apply_permutation` is deprecated, please use `tensor.index_select(dim, permutation)` instead
  h = apply_permutation(hx[0], permutation)
/opt/homebrew/Caskroom/miniconda/base/envs/kagi-demo/lib/python3.9/site-packages/diaparser/modules/lstm.py:87: FutureWarning: `apply_permutation` is deprecated, please use `tensor.index_select(dim, permutation)` instead
  c = apply_permutation(hx[1], permutation)


In [48]:
dataset.sentences

[1	what	_	_	_	_	0	root	_	_
 2	are	_	_	_	_	1	cop	_	_
 3	some	_	_	_	_	4	det	_	_
 4	ways	_	_	_	_	1	nsubj	_	_
 5	to	_	_	_	_	6	mark	_	_
 6	do	_	_	_	_	4	acl	_	_
 7	fast	_	_	_	_	9	amod	_	_
 8	query	_	_	_	_	9	compound	_	_
 9	reformulation	_	_	_	_	6	obj	_	_]